# Testing Script

In [8]:
import gurobipy as gp
from gurobipy import GRB

def optimizeTheater(seatSize, groupSize, cost):
    number_of_customers_in_group = groupSize
    number_of_seats = seatSize
    non_vax_cost = cost
    
    theater_seat_organizer = gp.Model()
    
    person_inGroup_atSeat = {}

    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            for seat in range(number_of_seats):

                person_inGroup_atSeat[customer_index, group_num, seat] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_atSeat" + str(seat))

    groupAccepted = {}
    for group_num in range(len(number_of_customers_in_group)):
        groupAccepted[group_num] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "group" + str(group_num) + "_admitted")


    objectiveFunction = gp.LinExpr()

    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            for seat in range(number_of_seats):
                objectiveFunction += person_inGroup_atSeat[customer_index, group_num, seat]   

    for group_num in range(len(number_of_customers_in_group)):
        objectiveFunction -= non_vax_cost * (1-groupAccepted[group_num])

    theater_seat_organizer.setObjective(objectiveFunction,  GRB.MAXIMIZE)
    
    # at most one person per seat
    for seat in range(number_of_seats):
        at_most_one_person_per_seat = gp.LinExpr()

        for group_num in range(len(number_of_customers_in_group)):
            for customer_index in range(0, number_of_customers_in_group[group_num]):
                at_most_one_person_per_seat += person_inGroup_atSeat[customer_index, group_num, seat]   
        theater_seat_organizer.addConstr(at_most_one_person_per_seat <= 1, name = "at_most_one_person_at_seat" + str(seat))

    # the whole group must be added
    for group_num in range(len(number_of_customers_in_group)):

        group_must_be_added = gp.LinExpr() # collects the sum of all the seats the group sits in
        is_group_accepted = gp.LinExpr(number_of_customers_in_group[group_num] * groupAccepted[group_num]) #determines if the group will be seated or not

        for customer_index in range(0, number_of_customers_in_group[group_num]):
            for seat in range(number_of_seats):
                group_must_be_added += person_inGroup_atSeat[customer_index, group_num, seat]   

        theater_seat_organizer.addConstr(group_must_be_added == is_group_accepted, name = "is_group_" + str(group_num) + "_accepted")

    # each person should be assigned to only one seat 
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            seats_per_person = gp.LinExpr()

            for seat in range(number_of_seats):
                seats_per_person += person_inGroup_atSeat[customer_index, group_num, seat] 

            theater_seat_organizer.addConstr(seats_per_person <= 1, name = "person" + str(customer_index) + "_group" + str(group_num) + "_uniqueSeat")
            
    # There must be an empty seat between groups 
    for seat in range(number_of_seats - 1):

        for group_num in range(len(number_of_customers_in_group)):
            for other_group_num in range(len(number_of_customers_in_group)):

                if not(group_num == other_group_num):    

                    for customer_index in range(0, number_of_customers_in_group[group_num]):
                        for customer2_index in range(0, number_of_customers_in_group[other_group_num]):

                            seat1 = person_inGroup_atSeat[customer_index, group_num, seat]
                            seat2 = person_inGroup_atSeat[customer2_index, other_group_num, seat+1]
                            label = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_andPerson" + str(customer2_index) + "+inGroup" + str(other_group_num) + "_cannotSitTogether"
                            theater_seat_organizer.addConstr( seat1 + seat2 <= 1, name = label)
                            
    theater_seat_organizer.optimize()
    
    seating_scheme = list()
    for seat in range(number_of_seats):
        seatFound = False
        for group_num in range(len(number_of_customers_in_group)):
            for customer_index in range(0, number_of_customers_in_group[group_num]):
                result = person_inGroup_atSeat[customer_index, group_num, seat]

                if( result.x == 1 ):
                    seatFound = True
                    print( "Seat" + str(seat) + ": person" + str(customer_index) + " fromGroup" + str(group_num))

                    seating_scheme.append((customer_index, group_num))

        if(not seatFound):
            print("Seat" + str(seat) + ": <empty>")
            seating_scheme.append(None)
            
    genericSpacing = list()
    last_group = -1
    for value in seating_scheme:
        if(type(value) == tuple and value[1] != last_group):
            genericSpacing.append((None,None))
            genericSpacing.append(value)
            last_group = value[1]


        elif(type(value) == tuple and value[1] == last_group):
            genericSpacing.append(value)

    genericSpacing.pop(0)

    while(genericSpacing[0][1] == genericSpacing[-1][1]):
        genericSpacing.append(genericSpacing[0])
        genericSpacing.pop(0)

    genericSpacing.pop(0)

    for members in genericSpacing:
        if not (members[0] == None):
            print("person " + str(members[0] + 1) + " from group " + str(members[1] + 1))
        else:
            print("<SPACE>")
            
    for group_num in range(len(groupAccepted)):
        if(groupAccepted[group_num].x != 1):
            print("Group " + str(group_num + 1) + " is excluded")
        else:
            print("Group " + str(group_num + 1) + " is accepted")

In [18]:
# Test Variables
number_of_customers_in_group = [[5,7,8,1,4,6,7,8,2,6,5,6,7,8,9,9,5,4,7,8,9]]
number_of_seats = [160]
non_vax_cost = [10]



In [19]:
for index, inputs in enumerate(number_of_customers_in_group):
    print("TEST CASE #" + str(index))
    
    optimizeTheater(number_of_seats[index], inputs, non_vax_cost[index])

TEST CASE #0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 2583426 rows, 20981 columns and 5229129 nonzeros
Model fingerprint: 0xdb60606c
Variable types: 0 continuous, 20981 integer (20981 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -210.0000000
Presolve removed 1 rows and 0 columns (presolve time = 5s) ...
Presolve removed 2489712 rows and 0 columns (presolve time = 10s) ...
Presolve removed 2499062 rows and 0 columns (presolve time = 15s) ...
Presolve removed 2508806 rows and 0 columns (presolve time = 20s) ...
Presolve removed 2518609 rows and 0 columns (presolve time = 25s) ...
Presolve removed 2531923 rows and 0 columns (presolve time = 30s) ...
Presolve removed 2532128 rows and 0 columns (presolve time = 36s) ...
Pr